In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv("data.csv", index_col=0, delimiter=";")
df["diff"] = df["Druhe_mereni"] - df["Prvni_mereni"]
df

In [ ]:
df_control = df.query("Skupina == 0")
df_exp = df.query("Skupina == 1")

In [ ]:
df_control.describe()

In [ ]:
df_exp.describe()

Statistical tests scipy: https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-tests


# Úkol 6: T-testy
Zkoumáme, zdali námi navržený kognitivní trénink vede ke zmírnění příznaků demence u seniorů s touto diagnózou. Jelikož kromě samotného programu zde můžou mít blahodárný účinek i nespecifické faktory (např. samotný čas strávený komunikací s pacientem), zvolili jsme design s kontrolní skupinou. Zatímco experimentální skupina dostávala kognitivní trénink, skupina kontrolní trávila čas jinými aktivitami. Pacienty nebylo možné do skupin rozlosovat, při rozdělování jsme však usilovali o vyrovnané skupiny. U každého pacienta byly před začátkem a po skončení programu vyšetřeny jeho kognitivní schopnosti pomocí testu. Nalezené počty bodů jsou uvedeny v tabulce níže (čím více bodů, tím lepší schopnosti); jedničkou je značena experimentální skupina, nulou kontrolní.

U sledovaných náhodných veličin předpokládejme normální rozdělení a shodu rozptylů mezi skupinami. Není-li řečeno jinak, uvádějte hodnotu statistiky t bez znaménka.

Instrukce k odeslání obrázku najdete v poznámce níže. Budete-li odesílat obrázek těsně před deadlinem, může se stát, že již nebudete mít možnost si jej v případě zamítnutí opravit.

=====================================================

Mezi pacienty zařazenými do kontrolní a experimentální skupiny existuje rozdíl již kognitivních schopnostech již před zahájením programu (v pretestu).

In [ ]:
a = df_control["Prvni_mereni"].values
b = df_exp["Prvni_mereni"].values

stats.ttest_ind(a, b, alternative="two-sided")

Po skončení programu skórují pacienti z experimentální skupiny v testu kognitivních schopností výše než pacienti z kontrolní skupiny.

greater’: the mean of the distribution underlying the first sample is greater than the mean of the distribution underlying the second sample.

In [ ]:
a = df_exp["Druhe_mereni"].values
b = df_control["Druhe_mereni"].values

stats.ttest_ind(a, b, alternative="greater")

Absolvování tréninku zvyšuje skóre získané v testu kognitivních schopností.

In [ ]:
a = df_exp["Druhe_mereni"].values
b = df_exp["Prvni_mereni"].values

stats.ttest_rel(a, b, alternative="greater")

Absolvování aktivit učených pro kontrolní skupinu zvyšuje skóre získané v testu kognitivních schopností.

In [ ]:
a = df_control["Druhe_mereni"].values
b = df_control["Prvni_mereni"].values

stats.ttest_rel(a, b, alternative="greater")

Velikost zlepšení kognitivních schopností je v experimentální skupině vyšší než ve skupině kontrolní

In [ ]:
a = df_exp["diff"].values
b = df_control["diff"].values

stats.ttest_ind(a, b, alternative="greater")

O kolik směrodatných odchylek se experimentální skupina zlepšila? K výpočtu míry účinku použijte směrodatnou odchylku z celého souboru v pretestu. (Jedná se o přesnější odhad, než kdybychom použili data jen z experimentální skupiny.)

In [ ]:
d_mean = np.mean(df_exp["diff"].values)
mu_0 = 0

std = np.std(df["Prvni_mereni"], ddof=1)

effect_size = (d_mean - mu_0) / std
effect_size

Vytvořte graf s 95% konfidenčními intervaly, z nějž můžeme přehledně vyčíst odpovědi na první 4 hypotézy a od oka srovnat míru zlepšení mezi skupinami. Nezapomeňte na popisky os a legendu. (Tento úkol možná bude vyžadovat přeorganizování datová tabulky, použití jiných softwarů a jistou dávku kreativity. Výsledek by mohl vypadat třeba takto, ale fantazii se meze nekladou.)

In [ ]:
def get_mean_and_error(values, alpha):
    n = len(values)
    mean = np.mean(values)
    sem = stats.sem(values)
    q = 1-alpha/2
    error = stats.t.ppf(q=q, loc=0, scale=sem, df=n-1)
    interval_1 = stats.t.interval(alpha=1-alpha, loc=mean, scale=sem, df=n-1)
    interval_2 = (mean - error, mean+error)
    # print(f"{mean=}, {error=}, {interval_1=}, {interval_2=}")
    return mean, error

x = ["Pre-test", "Post-test"]

mean_1, error_1 = get_mean_and_error(df_control["Prvni_mereni"], alpha=0.05)
mean_2, error_2 = get_mean_and_error(df_control["Druhe_mereni"], alpha=0.05)

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Kontrolní',
    x=x, y=[mean_1, mean_2],
    error_y=dict(type='data', array=[error_1, error_2])
))

mean_1, error_1 = get_mean_and_error(df_exp["Prvni_mereni"], alpha=0.05)
mean_2, error_2 = get_mean_and_error(df_exp["Druhe_mereni"], alpha=0.05)

fig.add_trace(go.Bar(
    name='Experimentální',
    x=x, y=[mean_1, mean_2],
    error_y=dict(type='data', array=[error_1, error_2])
))
fig.update_layout(
    barmode='group', 
    yaxis_title="Počet bodů v kognitivním testu",
    legend_title="Skupina",
    title="Vliv kognitivního tréninku na kognitivní schopnosti <br>u seniorů s demencí"
)
fig.show()
fig.write_image("06_chart.png",  scale=2)